# Multitexture OBJ Support in PyTorch3d

- Support for multitextured OBJs is an important capability for applied deep learning on 3d meshes and point clouds.
- A key usecase is applying point cloud segmentation to mesh segmentation problems or embedding labels into obj meshes as outputs of a deep learning pipeline.
- This notebook introduces feature enhancements to PyTorch3D that provides multitexture OBJ support for saving, reading, and manipulating OBJs with multiple textures.

## Summary of new or ehanced functions
1. pytorch3d.ops.sample_points_from_obj() is a new function that allows a user to sample at least one point from all faces with a new auto sampling feature that determines a number of points to sample. Although a new function, sample_points_from_obj repackages existing PyTorch3D functionality from pytorch3d.ops.sample_points_from_meshes(). The enhancements in sample_points_from_obj importantly allow for sampling all faces with a minimum sampling factor and point to face index mappers tensor that allows a link each point to its origin face.
2. pytorch3d.ops.sample_points_from_meshes() is modified to enable sample_points_from_obj() by grouping key capabilities into helper functions that can be leveraged in sample_points_from_obj. Further, sample_points_from_meshes is modified slightly to optionally return point-to-face index mappers which can allow a user to recover the face for each point; however, modifications to sample_points_from_meshes are kept minimal and do not provide the new features in sample_points_from_obj.
3. pytorch3d.io.obj_io.subset_obj() is a new function that allows a user to subset an obj mesh based on selected face indices. For example, if a workflow predicts a per-face classification, this function can be used to subset the mesh for only those faces. 
4. pytorch3d.io.obj_io.save_obj() and pytorch3d.io.obj_io.load_obj_as_meshes() provide integrated multi-texture obj support to allow users to read and process all available textures; PyTorch3D previously only reads the first texture in an input obj file with multiple textures which can lead to undesirable texture sampling and output.
5. pytorch3d.utils.obj_utils provides common utilities for use in both pytorch3d.ops and pytorch3d.io.obj_io such as consolidating obj validation (_validate_obj) and core implementation for subsetting obj data.

### 0. Install and Import modules
Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
import os
import sys
import torch
# install the pytorch3d fork
!pip install 'git+https://github.com/ArcGIS/pytorch3d.git@multitexture-obj-point-sampler'

In [ ]:
# import open source tools
import os
import numpy as np
import torch
from collections import Counter

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# import pytorch3d tools
from pytorch3d.structures import Pointclouds, join_meshes_as_scene
from pytorch3d.ops import  sample_points_from_obj # a new function
from pytorch3d.io import (
    load_obj,
    subset_obj, # a new function
    save_obj,
    load_objs_as_meshes
    )

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras, 
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    FoVPerspectiveCameras,
    PointLights
)

In [ ]:
# download and make directories for IO or create as needed in local directories
!mkdir -p data/cow_mesh
!mkdir -p data/output
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl
!wget -P data/cow_mesh https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png

DATA_DIR = os.path.join('data/cow_mesh')
OUTPUT_DIR = os.path.join('data/output')

In [ ]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
print(device)

### 1. Load OBJ and Use sample_points_from_obj

The following cells read and write an OBJ, the cow mesh.
It renders the cow mesh as in other tutorials except it ensures that at least one point is sampled from each face.
1. Since all faces are represented, we can apply point cloud classification or segmentatio to the mesh.
2. sample_points_from_obj leverages modifications in the current sample_points_from_meshes 
3. However, unlike sample_points_from_meshes, sample_points_from_obj only samples for one obj at a time.

In [ ]:
# specify the input obj; in this case the tuturial data's cow mesh
f = os.path.join(DATA_DIR, 'cow.obj')

In [ ]:
# load the obj into memory
obj = load_obj(
    f=f,
    texture_wrap=None,
    create_texture_atlas=True,
    texture_atlas_size=8, 
    device=device
)

# use sampler to return points from the obj using a specified number of points
points, normals, textures, mappers = sample_points_from_obj(
    verts=obj[0],
    faces=obj[1].verts_idx,
    verts_uvs=obj[2].verts_uvs,
    faces_uvs=obj[1].textures_idx,
    texture_images=obj[2].texture_images,
    materials_idx=obj[1].materials_idx,
    texture_atlas=obj[2].texture_atlas,
    num_samples=None,
    min_sampling_factor=100,
    sample_all_faces=True,
    return_mappers=True, 
    return_textures=True, 
    return_normals=True,
    use_texture_atlas=True
)

# squeeze batches out of all tensors
(points, normals, textures, mappers) = (
    points.squeeze(0),
    normals.squeeze(0),
    textures.squeeze(0),
    mappers.squeeze(0)
)

### 2. Render the cow mesh as provided by PyTorch3d Tutorials

A key new feature is the ability to set num_samples to None and provide a scalar value as a point sampling factor to provide relatively larger or smaller point cloud densities during sample_points_from_obj. In this case, a min_sampling_factor of 100 (very dense) is provided. This feature can be helpful if it is not clear exactly how many points should be sampled to produce a point cloud with sufficient density for deep learning. Specifically, auto sampling produces samples proportionally to face area while considering the number of faces.

In [ ]:
# set up a point cloud renderer according to PyTorch3D tutorials
# https://pytorch3d.org/tutorials/render_colored_points
R, T = look_at_view_transform(2.7, 0, 180)
cameras = FoVOrthographicCameras(
    device=device,
    R=R,
    T=T,
    znear=0.01
)
raster_settings = PointsRasterizationSettings(
    image_size=256, 
    radius = 0.003,
    points_per_pixel = 10
)
rasterizer = PointsRasterizer(
    cameras=cameras,
    raster_settings=raster_settings
)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor((128, 128, 128))
)

In [ ]:
point_cloud = Pointclouds(
    points=[points],
    features=[textures],
    normals=[normals])
images = renderer(point_cloud)
plt.figure(figsize=(6, 6))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.title("A rendered 3D Cow Mesh as a Point Cloud with PyTorch3D\nSampled with at least one point per face")
plt.axis("off")

### 3. Apply a Point Cloud Part Segmentation Workflow

In this toy example, we use KMeans as a stand-in for a deep learning based point cloud segmentation workflow. Importantly, we can demonstrate how a mesh segmentation problem can be re-framed as a point cloud segmentation problem, if desired. With the new features, this type of re-framing is possible since all faces are represented in the sample. Importantly, this feature can allow researchers to compare and contrast methods for classification and segmentation that apply point cloud or mesh based workflows on the same input meshes.

In [ ]:
# in this example we use KMeans clustering as a simplistic way to segment this mean
n_clusters = 3

scaler = StandardScaler()
points_scaled = scaler.fit_transform(points.cpu().numpy())

model = KMeans(n_clusters=n_clusters)
model.fit(points_scaled)
clusters = model.labels_

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.axis('off')
ax.scatter(
    points_scaled[..., 2], points_scaled[..., 0], points_scaled[..., 1],
    c=clusters)
plt.title(f'Finding {n_clusters}x Clusters on a 3D Cow Mesh with KMeans\nAs a Pseudo Point Class Prediction')
plt.show()

# the KMeans clusters as predictions per point could be replaced by a better Deep Learning approach
assert points.shape[0] == clusters.shape[0]

In [ ]:
# at this point we have a predicted class per point and can apply those classifications to the origin mesh
def majority_vote(x: np.ndarray) -> int:
    """A helper function to count and return the most occuring value in an array."""
    if x.shape[0] == 1:
        return x[0]
    else:
        if not np.any(x):
            return np.bincount(x).argmax()
        else:
            # for data structs containing negative values
            return Counter(x).most_common(1)[0][0]

face_classes = np.zeros(obj[1].verts_idx.shape[0])
_mappers = mappers.cpu().numpy()

for i in range(obj[1].verts_idx.shape[0]):
    mask = _mappers == i
    face_classes[i] = majority_vote(clusters[mask])

print(f'There are {obj[1].verts_idx.shape[0]} faces in the mesh and {np.unique(face_classes).shape[0]} classes')

### 4. Produce output OBJ meshes after segmentation

We can now combine subset_obj() and save_obj() to produce useful outputs after segmenting a mesh. In this case, we might be interested in writing a subsesh for each part. Optionally we can decide to re-use materials and have each subset reference the same material files to avoid writing the same images. 

In [ ]:
# given a new class or texture for each face, we can apply classes to the mesh
unique_classes = np.unique(face_classes).astype(int)
subset_obj_files = []

# for each face class, return a subset obj of only those faces
for unique_class in unique_classes:
    # return the faces that belong to the current class as an index
    faces_to_subset = np.flatnonzero(face_classes == unique_class)
    faces_to_subset = torch.from_numpy(faces_to_subset).to(device)
    
    # apply the face indices to subset the obj
    obj_subset = subset_obj(
        obj=obj,
        faces_to_subset=faces_to_subset,
        device=torch.device('cpu')
    )

    image_name_kwargs = dict(
        reuse_material_files=False # If true will reuse the same material file for subsequet objs
    )

    obj_name = f"cow_subset_{unique_class}"
    obj_f = os.path.join(OUTPUT_DIR, f"{obj_name}.obj")
    texture_images = {}

    for k, v in obj_subset[2].texture_images.items():
        random_color = np.random.uniform(0, 1, 3)

        texture_images[f'{k}_class_{unique_class}'] = torch.from_numpy(np.full(v.shape, random_color))
    # save each of the obj subsets as individual objs
    save_obj(
        f=obj_f,
        verts=obj_subset[0],
        faces=obj_subset[1].verts_idx,
        verts_uvs=obj_subset[2].verts_uvs,
        faces_uvs=obj_subset[1].textures_idx,
        texture_images=texture_images,
        materials_idx=obj_subset[1].materials_idx,
        image_name_kwargs=image_name_kwargs,
    )
    # print(obj_subset[2].texture_images)
    subset_obj_files.append(obj_f)


### 5. Read and validate multitexture OBJ support

To validate the results, we can also use load_objs_as_meshes with multitexture OBJ support to read in the previous outputs as a single mesh. Notice that the result is the same as the prior output; this means we were able to deconstruct and reconstruct the OBJ mesh by semantic segmentation.

In [ ]:
# set up rendering for a meshes object according to PyTorch3d Tutorials
# https://pytorch3d.org/tutorials/fit_textured_mesh
R, T = look_at_view_transform(2.7, 0, 180)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

raster_settings = RasterizationSettings(
    image_size=256,
    blur_radius=0.0,
    faces_per_pixel=1,
)

lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights),
)

# read in each of the subsets as a batch of individual meshes
mesh_from_subsets = load_objs_as_meshes(
    files=subset_obj_files,
    texture_wrap=None,
    device=device
)

# join the input meshes as a scene
mesh = join_meshes_as_scene(mesh_from_subsets)
# render the image: we should have the input cow
images = renderer(mesh)
plt.figure(figsize=(6, 6))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")
plt.tight_layout()
plt.title("A 3D Cow Mesh Reconstructed From Subsets\nColored by Predicted Class of Points Sampled from Each Face")
plt.show()


### 6. Write OBJ with Multiple Textures

Finally, a key new feature can allow users to write to disc a single OBJ with multiple textures and with full or high precision vertices. In this case, each material is represented by a random color meant to visualize classes in the mesh. In this example, we use KMeans to provide an arbitrary classification for each face via their sampled points; however, in complex cases, KMeans could be replaced with a state of the art model that produces a per-point classification or segmentation result. 

In the following cells, we manually merge an OBJ mesh so that the original cow mesh, which had only one material/texture, now has at least three materials (one for each cluster or class). Then, we exercise the new feature of save_obj to save this obj to disc and read it back to provide the expected mesh. Without this new feature, it is only possible to read the first material in a list of materials. For outputs, a user *can* write a Meshes object with multiple textures as a single concatenated scene of materials; however, this method does not generalize well. The reason for this is that if there are many large textures, as is the case for city or regional meshes, PyTorch3D will run into memory IO limitations in writing single, large images to disc.

#### 6.a. Working with multitexture OBJ IO 

In [ ]:
# we can leverage multitexture obj support for saving meshes to disc with multiple textures

# read in the obj from disc into memory subset objs by class but accumulate them into a single obj file
offset = 0
verts, faces, verts_uvs, faces_uvs, materials_idx = [], [], [], [], []
texture_images = {}
material_counter = 0

for f in subset_obj_files:
    obj_subset = load_obj(
        f=f,
        texture_wrap=None, 
    )
    # accumulate and offset tensors 
    verts.append(obj_subset[0])
    faces.append(obj_subset[1].verts_idx + offset)
    verts_uvs.append(obj_subset[2].verts_uvs)
    faces_uvs.append(obj_subset[1].textures_idx)
    
    for k, v in obj_subset[2].texture_images.items():
        # this example is a special case where we can simply accumulate textures
        texture_images[k] = v
        # in a general case, we need to handle seeing the same texture more than once
            
    materials_idx.append(obj_subset[1].materials_idx + material_counter)
    offset += obj_subset[0].shape[0]
    # in the general case, we would need to adjust how materials_idx are offset
    material_counter += 1

obj_name = f"cow_multitexture_subset"
obj_f = os.path.join(OUTPUT_DIR, f"{obj_name}.obj")

# save the resulting obj with multiple textures into a mesh
save_obj(
    f=obj_f,
    verts=torch.cat(verts),
    faces=torch.cat(faces),
    verts_uvs=torch.cat(verts_uvs),
    faces_uvs=torch.cat(faces_uvs),
    texture_images=texture_images,
    materials_idx=torch.cat(materials_idx),
    image_name_kwargs=image_name_kwargs,
)


mesh = load_objs_as_meshes(
    files=[obj_f],
    device=device
)
images = renderer(mesh)
plt.figure(figsize=(6, 6))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off")
plt.tight_layout()
plt.title("A 3D Cow Mesh With Multitexture IO with OBJs and Meshes")
plt.show()


#### 6.b. Working with Real World Coordinate Geometries with High Precision

In [ ]:
# using the same obj output from the preceding cells, test high precision
obj = load_obj(
    f=obj_f,
)

# we can simulate the case of real-world coordinate geometries by adding a large value to x y and z
verts_fp32 = obj[0] + 5000000
print(verts_fp32)
# as we can see from the print out, float32 starts rounding values which might represent real coordinates
# this effect compounds if sampling from this mesh since the points also lack precision

In [ ]:
# using the same obj output from the preceding cells, test high precision
obj = load_obj(
    f=obj_f,
    high_precision=True
)

# we can simulate the case of real-world coordinate geometries by adding a large value to x y and z
verts_fp64 = obj[0] + 5000000
print(verts_fp64)
# in this case, since the tensor can handle full precision values, the data is not rounded
# for high precision verts, the resulting sampled points produce ideal coordinates

## In Summary:

- This notebook demonstrates multitexture OBJ IO support and enhanced point cloud sampling with PyTorch3D
- IO for multitextured OBJs is a new feature in this fork of PyTorch3d.
- A key use case is applying point cloud segmentation to mesh segmentation problems.
- *sample_points_from_obj()* allows for sampling at least one point from all faces with auto sampling to determine number of points to sample in addition to providing an index of mappers to recover each point's face origin.
- *sample_points_from_meshes()* is modified to allow code-reuse for sample_points_from_obj() and to provide mappers.
- KMeans clustering is used here to represent a point cloud segmentation result but could be replaced by a state of the art point segmentation model.
- The point classifications from KMeans are applied to the origin faces in the mesh as obj subsets with *subset_obj()* and saved as both individual obj meshes and composite meshes with multiple textures.
- The results of each input and output is rendered using PyTorch3d tutorial methods for point clouds and meshes and demonstrates how modifications to *save_obj()* and *load_obj()* provide integrated multitexture obj support.
- Lastly, we provide a feature to support obj IO with float64 or *high_precision* which allows for flexiblity in workin with real world coordinate geometries such as those found in building and city-scape meshes.
